In [1]:
import ipywidgets as widgets
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import math

from ipywidgets           import Button, Layout, ButtonStyle, VBox, HBox
from IPython.display      import display
from matplotlib.animation import FuncAnimation

import rospy
import actionlib
import actionlib.msg
import assignment_2_2023.msg

from geometry_msgs.msg     import Point, Pose, Twist
from assignment_2_2023.msg import Cstm_vel
from nav_msgs.msg          import Odometry
from assignment_2_2023.srv import Last_goal, Last_goalResponse
from assignment_2_2023.msg import PlanningActionGoal
from assignment_2_2023.srv import Dst_avSpeed, Dst_avSpeedResponse
from std_msgs.msg          import String
from actionlib_msgs.msg    import GoalID

import math
import time
import numpy as np

rospy.init_node("node_a_notebook")
action_client = actionlib.SimpleActionClient('/reaching_goal', assignment_2_2023.msg.PlanningAction)
action_client.wait_for_server()

True

In [2]:
class Visualizer():
    def __init__(self):
        # definisco 2 elementi per plottare 
        self.figure, self.ax = plt.subplots(1, 2)
        
        self.targ, = self.ax[0].plot([], [], 'ro')
        self.line, = self.ax[0].plot([], [], 'bo')
        
        self.xTarg, self.yTarg = [], []
        self.xPose, self.yPose = [], []
        
        self.barVal = [0, 0, 0]
        self.bar = self.ax[1].bar(['Sent', 'Cancel', 'Success'], self.barVal, color=['blue', 'red', 'green']) 
        
    def init_plot(self):
        self.ax[0].set_xlim(-10, 10)
        self.ax[0].set_ylim(-10, 10)
        return self.targ, self.line, self.bar
    
    def update_plot(self, frame):
        self.targ.set_data(self.xTarg, self.yTarg)
        self.line.set_data(self.xPose, self.yPose)
        for i, b in enumerate(self.bar):  # Update the bar plot
            b.set_height(self.barVal[i])
            self.ax[1].set_ylim(0, max(self.barVal) + 1)
        return self.line, self.targ, self.bar
    
    def save_targhet(self, x, y):
        self.xTarg.append(x)
        self.yTarg.append(y)
        
    def update_position_callback(self, msg):
        self.xPose.append(msg.x)
        self.yPose.append(msg.y)
        
    
    def add_sent(self):
        self.barVal[0] += 1
        
    def add_cancel(self):
        self.barVal[1] += 1
        #svuoto le liste della posizione 
        self.xPose.clear()
        self.yPose.clear()
        
    def add_succes(self):
        self.barVal[2] += 1
        #svuoto le liste della posizione 
        self.xPose.clear()
        self.yPose.clear()
        
        
def get_server_feedback(feedback):
    if feedback.stat == "Target reached!":
        visual.add_succes()

In [3]:
%matplotlib widget
#interface widgets and logic 
x_cord = widgets.IntSlider(value=0, min=-10, max=10)
y_cord = widgets.IntSlider(value=0, min=-10, max=10)

send_bttn = Button(description='Send Goal', layout=Layout(width='auto', align="center"))
canc_bttn = Button(description='Cancel Goal', layout=Layout(width='auto', align="center"))

UI_layout = HBox([VBox([x_cord, y_cord]), send_bttn, canc_bttn])

def send_goal(b):
    goal = assignment_2_2023.msg.PlanningGoal()
    goal.target_pose.pose.position.x = float(x_cord.value)
    goal.target_pose.pose.position.y = float(y_cord.value)
    
    visual.add_sent()
    visual.save_targhet(x_cord.value, y_cord.value)
    
    action_client.send_goal(goal, feedback_cb=get_server_feedback)

    
def cancel_goal(b):
    if action_client.get_state() == actionlib.GoalStatus.SUCCEEDED:
        print("Goal aleady reached")
    else:
        action_client.cancel_goal()
        visual.add_cancel()
        print("Goal canceled")


In [4]:
#node logic
visual = Visualizer()
subscriber = rospy.Subscriber('/custom_vel', Cstm_vel, visual.update_position_callback)


ani = FuncAnimation(visual.figure, visual.update_plot, init_func=visual.init_plot,frames=range(100), blit=True)
plt.show()
display(UI_layout)
send_bttn.on_click(send_goal)
canc_bttn.on_click(cancel_goal)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Goal canceled
